<a href="https://colab.research.google.com/github/yodaboop/Capstone-Project/blob/main/Capstone_RAG_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ----------------------------------------------------
# 1. SETUP AND DEPENDENCIES (LangChain + OpenAI)
# ----------------------------------------------------
!pip install -q langchain langchain-core langchain-community langchain-openai langchain-text-splitters faiss-cpu pypdf

import os
import sys
from google.colab import files, userdata

# LangChain + OpenAI imports
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

# LCEL components (modern LangChain, no langchain.chains)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

# ----------------------------------------------------
# 1.1 Load your OpenAI API key from Colab secrets
# ----------------------------------------------------

api_key = userdata.get("OpenAI_API_Edu")  # 👈 this is your existing secret name

if not api_key:
    raise ValueError("❌ No OpenAI_API_Edu found in Colab secrets. Add it under 'Secrets' first.")

os.environ["OPENAI_API_KEY"] = api_key  # used by langchain_openai

print("✅ OPENAI_API_KEY loaded from Colab secrets.")

# ----------------------------------------------------
# 2. UPLOAD THE PDF FILE
# ----------------------------------------------------
print("\nPlease upload the PDF file you want to use for RAG.")
try:
    uploaded = files.upload()
except Exception as e:
    print(f"File upload failed: {e}")
    sys.exit()

if not uploaded:
    print("\n⚠️ ERROR: No file was uploaded. Please try again.")
    sys.exit()

FILE_NAME = list(uploaded.keys())[0]
print(f"\n✅ SUCCESS: File '{FILE_NAME}' uploaded and ready for processing.")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.8/475.8 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
✅ OPENAI_API_KEY loaded from Colab secrets.

Please upload the PDF file you want to use for RAG.


Saving NexusTech Employee Guide.pdf to NexusTech Employee Guide.pdf

✅ SUCCESS: File 'NexusTech Employee Guide.pdf' uploaded and ready for processing.


In [ ]:
# ----------------------------------------------------
# 3. INDEXING AND RAG CHAIN (LangChain + OpenAI)
# ----------------------------------------------------

# 3.1 Load the PDF into documents

# Check if the file is actually a PDF before attempting to load
import pypdf # This import is added here, as it might not be explicitly imported in this cell

if not FILE_NAME.lower().endswith(".pdf"):
    print(f"❌ Error: Expected a PDF file, but '{FILE_NAME}' is not a PDF. Please upload a PDF file.")
    sys.exit()

try:
    loader = PyPDFLoader(FILE_NAME)
    documents = loader.load()
    print(f"✅ Loaded {len(documents)} page(s) from {FILE_NAME}.")
except pypdf.errors.PdfStreamError as e:
    print(f"❌ Error loading PDF: The uploaded file '{FILE_NAME}' appears to be corrupted or not a valid PDF. Please ensure you upload a correct PDF file.")
    sys.exit()
except Exception as e:
    print(f"❌ An unexpected error occurred while loading PDF: {e}")
    sys.exit()

# 3.2 Split into smaller chunks for better retrieval
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
docs = text_splitter.split_documents(documents)
print(f"✅ Created {len(docs)} chunks.")

# 3.3 Create embeddings + FAISS vector store
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(docs, embeddings)

# 3.4 Turn the vector store into a retriever
retriever = vectorstore.as_retriever(search_kwargs={"k": 4})

# 3.5 Define the OpenAI chat model
llm = ChatOpenAI(
    model="gpt-4o-mini",   # or "gpt-4.1-mini", "gpt-3.5-turbo", etc
    temperature=0
)

# 3.6 Define the prompt + RAG chain using LCEL
prompt = ChatPromptTemplate.from_template(
    """You are a helpful assistant that answers questions using the provided context.

Context:
{context}

Question:
{input}

If the answer is not in the context, say you don't know based on the document, you will transfer the conversation to human agent to helps user"""
)

def format_docs(docs):
    return "\n\n".join(d.page_content for d in docs)

# RAG chain:
#   input (string question)
#   └─> retriever -> docs -> format_docs -> {context}
#   └─> {context, input} -> prompt -> llm -> text
rag_chain = (
    {
        "context": retriever | RunnableLambda(format_docs),
        "input": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

print("✅ LangChain RAG chain initialized (OpenAI + FAISS).")


✅ Loaded 3 page(s) from NexusTech Employee Guide.pdf.
✅ Created 7 chunks.
✅ LangChain RAG chain initialized (OpenAI + FAISS).


In [ ]:
# ----------------------------------------------------
# 4. ASK QUESTIONS TO THE RAG AGENT
# ----------------------------------------------------

def ask(query: str):
    print("\n" + "="*80)
    print("QUESTION:", query)
    print("="*80)
    answer = rag_chain.invoke(query)   # LCEL chains accept raw string input
    print("ANSWER:\n", answer)

# Example questions (adapt to your PDF)
ask("What is the password for the Nexus-Employees Wi-Fi?")
ask("Where is the main IT Support Desk located?")
ask("Summarize the main onboarding steps for new employees.")



QUESTION: What is the password for the Nexus-Employees Wi-Fi?
ANSWER:
 The password for the Nexus-Employees Wi-Fi is SecureNet#2025.

QUESTION: Where is the main IT Support Desk located?
ANSWER:
 The main IT Support Desk is located on the Second Floor, in Room S-01, next to the elevators.

QUESTION: Summarize the main onboarding steps for new employees.
ANSWER:
 The main onboarding steps for new employees are as follows:

1. **Check-in**: New employees must check in with Human Resources on the Ground Floor to receive their ID badge and access card.
2. **Complete Initial Paperwork**: All initial paperwork must be completed within the first five business days.
3. **Pick Up IT Equipment**: Pre-configured IT equipment can be picked up from the IT Support Desk on the Second Floor (Room S-01).
4. **Attend Orientation**: A mandatory orientation is held every Monday at 10:00 AM in the Client Demo Room (T-04).
